In [1]:
# Complete Website Summarization Code - Run this entire cell!

import requests
from bs4 import BeautifulSoup
import ollama

def fetch_website_content(url):
    """Fetch and extract text content from a website"""
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        for element in soup(['script', 'style', 'nav', 'footer', 'header', 'aside']):
            element.decompose()
        
        text = soup.get_text(separator='\n', strip=True)
        lines = [line.strip() for line in text.splitlines() if line.strip()]
        clean_text = '\n'.join(lines)
        
        return clean_text
    except Exception as e:
        return f"Error fetching website: {str(e)}"

def summarize_with_ollama(text, model='llama3.2', max_length=4000):
    """Summarize text using Ollama"""
    try:
        text_to_summarize = text[:max_length]
        
        prompt = f"""Please provide a clear and concise summary of the following text:

{text_to_summarize}

Summary:"""
        
        response = ollama.chat(
            model=model,
            messages=[{'role': 'user', 'content': prompt}]
        )
        
        return response['message']['content']
    except Exception as e:
        return f"Error with Ollama: {str(e)}"

def summarize_website(url, model='llama3.2'):
    """Complete pipeline to fetch and summarize a website"""
    print(f"Fetching content from: {url}")
    print("-" * 60)
    
    content = fetch_website_content(url)
    
    if content.startswith("Error"):
        print(content)
        return None
    
    print(f"✓ Content fetched!")
    print(f"✓ Content length: {len(content)} characters")
    print("-" * 60)
    print(f"\nGenerating summary...")
    
    summary = summarize_with_ollama(content, model=model)
    
    print("\n" + "=" * 60)
    print("SUMMARY")
    print("=" * 60)
    print(summary)
    print("=" * 60)
    
    return {'url': url, 'content': content, 'summary': summary}

# TEST IT - Change this URL to any website you want
url = "https://www.google.com/"
result = summarize_website(url)

Fetching content from: https://www.google.com/
------------------------------------------------------------
✓ Content fetched!
✓ Content length: 138 characters
------------------------------------------------------------

Generating summary...

SUMMARY
The text appears to be a simplified version of the Google homepage, likely translated into Nepali. The key points can be summarized as:

* Google is available in English and possibly other languages.
* It offers advanced search features (अन्नत खोज).
* The website is available at google.com.np (a domain specific to Nepal).
* Copyright notice states 2026 (Google App nhare).

Note that the text is quite brief and may be a truncated version of the actual Google homepage.
